### Exercise 3.1

In [2]:
import numpy as np
from MDP import *

# Define states and actions.
num_states = 6
states = np.arange(num_states)
actions = np.array([0, 1])  # Actions: 0 and 1
num_actions = len(actions)
P = 25 

# Define transition probabilities.
Q = np.array([
    [0.9, 0.1, 0.0, 0.0, 0.0],   
    [0.0, 0.8, 0.1, 0.05, 0.05], 
    [0.0, 0.0, 0.7, 0.1, 0.2],   
    [0.0, 0.0, 0.0, 0.5, 0.5],   
])

# Initialize the transitions array.
transitions = np.zeros((num_states, num_actions, num_states))

for s in states:
    if s == 0:
        # From state 0, any action leads to state 1 with probability 1.0
        for a in range(num_actions):
            transitions[s, a, 1] = 1.0
    elif s == 5:
        # From state 5, any action leads to state 0 with probability 1.0
        for a in range(num_actions):
            transitions[s, a, 0] = 1.0
    else:
        for a in range(num_actions):
            if a == 0:
                # Action 0: transitions defined by Q
                transitions[s, a, 1:6] = Q[s-1]
            else:
                # Action 1: transitions to state 1 with probability 1.0
                transitions[s, a, 1] = 1.0

# Define rewards.
rewards = np.zeros((num_states, num_actions))
c_p = [0, 7, 7, 5]  # Rewards for action 1 in states 1 to 4

for s in states:
    for a in range(num_actions):
        if s == 0:
            rewards[s, a] = 0
        elif s == 5:
            rewards[s, a] = -10
        elif a == 0:
            rewards[s, a] = P
        elif a == 1:
            rewards[s, a] = -c_p[s-1]

# Set the time horizon.
horizon = 7

# Initialize and solve the MDP.
mdp = FiniteHorizonMDP(num_states, num_actions, transitions, rewards, horizon)
V, policy, dV = mdp.solve()

# Print the results.
for t in range(horizon):
    print(f"Time {t}:")
    print("Values:", V[t])
    print("Policy:", policy[t])
    print("Value per step:", dV[t])

ihmdp = InfiniteHorizonMDP(num_states, num_actions, transitions, rewards)
V_v, policy_v = ihmdp.value_iteration()
print("Optimal Value:")
print(V_v)
print("Optimal Policy:")
print(policy_v)

Time 0:
Values: [0. 0. 0. 0. 0. 0.]
Policy: [0 0 0 0 0 0]
Value per step: [0. 0. 0. 0. 0. 0.]
Time 1:
Values: [  0.  25.  25.  25.  25. -10.]
Policy: [0 0 0 0 0 0]
Value per step: [  0.  25.  25.  25.  25. -10.]
Time 2:
Values: [ 25.    50.    48.25  43.    32.5  -10.  ]
Policy: [0 0 0 0 0 0]
Value per step: [25.   25.   23.25 18.    7.5   0.  ]
Time 3:
Values: [50.    74.825 69.025 56.35  45.    15.   ]
Policy: [0 0 0 0 1 0]
Value per step: [25.    24.825 20.775 13.35  12.5   25.   ]
Time 4:
Values: [74.825 99.245 88.855 71.945 69.825 40.   ]
Policy: [0 0 0 0 1 0]
Value per step: [24.825 24.42  19.83  15.595 24.825 25.   ]
Time 5:
Values: [ 99.245   123.206   108.76975  92.245    94.245    64.825  ]
Policy: [0 0 0 1 1 0]
Value per step: [24.42    23.961   19.91475 20.3     24.42    24.825  ]
Time 6:
Values: [123.206    146.762375 129.1938   116.206    118.206     89.245   ]
Policy: [0 0 0 1 1 0]
Value per step: [23.961    23.556375 20.42405  23.961    23.961    24.42    ]
Iteration 1:

### Exercise 3.2

In [3]:
import numpy as np
from scipy.stats import binom
from MDP import *

# Define parameters.
num_of_states = 6  # Possible inventory levels from 0 to 5.
num_of_actions = 6  # Possible order quantities from 0 to 5.
states = np.arange(num_of_states)
actions = np.arange(num_of_actions)
B = 5  # Maximum inventory level and maximum demand.
K = 4  # Fixed ordering cost
c = 2  # Unit ordering cost
h = 1  # Holding cost per unit
p = 8  # Price per unit sold (revenue)
b = 2  # Lost sale cost per unit
demand_prob = 0.6  # Probability parameter for binomial distribution

# Set the time horizon.
horizon = 11

# Define transition probabilities and rewards.
num_states = num_of_states
num_actions = num_of_actions

transitions = np.zeros((num_states, num_actions, num_states))
rewards = np.zeros((num_states, num_actions))

# Precompute demand probabilities.
demand_probs = np.array([binom.pmf(k, B, demand_prob) for k in range(B + 1)])

for s in states:
    for a in actions:
        # Compute new inventory level after ordering.
        inventory_level = min(B, s + a)
        max_possible_sales = min(B, inventory_level)  # Can't sell more than demand or inventory

        # Compute the probability of transitioning to each next state.
        for next_s in states:
            for demand in range(B + 1):
                # The next state is determined by the inventory level minus demand, bounded between 0 and B.
                next_inventory = max(0, inventory_level - demand)
                if next_inventory == next_s:
                    prob = demand_probs[demand]
                    transitions[s, a, next_s] += prob

        # Normalize probabilities to ensure they sum to 1.
        transitions[s, a, :] /= transitions[s, a, :].sum()

        # Compute expected immediate reward for state s and action a.
        # Rewards include revenue from sales minus costs.
        ordering_cost = K * (a > 0) + c * a
        expected_holding_cost = 0
        expected_lost_sale_cost = 0
        expected_revenue = 0

        for demand in range(B + 1):
            prob = demand_probs[demand]
            sales = min(inventory_level, demand)
            leftover_inventory = inventory_level - sales
            unmet_demand = max(0, demand - inventory_level)

            holding_cost = h * leftover_inventory
            lost_sale_cost = b * unmet_demand
            revenue = p * sales

            expected_holding_cost += prob * holding_cost
            expected_lost_sale_cost += prob * lost_sale_cost
            expected_revenue += prob * revenue

        total_expected_reward = (
            expected_revenue
            - ordering_cost
            - expected_holding_cost
            - expected_lost_sale_cost
        )

        rewards[s, a] = total_expected_reward

# Initialize and solve the MDP.
mdp = FiniteHorizonMDP(num_states, num_actions, transitions, rewards, horizon)
V, policy, dV = mdp.solve()


# Print the results.

for t in range(horizon):
    print(f"Time {t}:")
    print("Values:", V[t])
    print("Policy:", policy[t])
    print("Value per step:", dV[t])


ihmdp = InfiniteHorizonMDP(num_states, num_actions, transitions, rewards)
V_v, policy_v = ihmdp.value_iteration_discount()

# Print the results.
print("Optimal Value Function:")
print(V_v)
print("Optimal Policy:")
print(policy_v)

V_p, policy_p = ihmdp.policy_iteration()
print("Optimal Value Function:")
print(V_p)
print("Optimal Policy:")
print(policy_p)

Time 0:
Values: [0. 0. 0. 0. 0. 0.]
Policy: [0 0 0 0 0 0]
Value per step: [0. 0. 0. 0. 0. 0.]
Time 1:
Values: [10.14464 12.14464 14.14464 19.43808 22.14464 22.     ]
Policy: [4 3 2 0 0 0]
Value per step: [10.14464 12.14464 14.14464 19.43808 22.14464 22.     ]
Time 2:
Values: [23.22964746 25.22964746 27.22964746 30.44588483 34.73869619 37.22964746]
Policy: [5 4 3 0 0 0]
Value per step: [13.08500746 13.08500746 13.08500746 11.00780483 12.59405619 15.22964746]
Time 3:
Values: [35.82032349 37.82032349 39.82032349 43.50962173 47.65914715 49.82032349]
Policy: [5 4 3 0 0 0]
Value per step: [12.59067603 12.59067603 12.59067603 13.06373691 12.92045096 12.59067603]
Time 4:
Values: [48.54531946 50.54531946 52.54531946 56.10514191 60.28953115 62.54531946]
Policy: [5 4 3 0 0 0]
Value per step: [12.72499597 12.72499597 12.72499597 12.59552017 12.630384   12.72499597]
Time 5:
Values: [61.23321801 63.23321801 65.23321801 68.82881205 73.00361456 75.23321801]
Policy: [5 4 3 0 0 0]
Value per step: [12.68

### Exercise 4.2 (Also applies to 3.3)

In [5]:
import numpy as np
from scipy.stats import binom
from MDP import *

# Define parameters.
M = 5  # Maximum spare parts the repairman can carry
N = 5  # Number of towns
num_states = (M + 1) * N  # States 0 to M
num_actions = 2  # Actions: 0 (do not replenish), 1 (replenish to M)
max_demand = 2  # Maximum demand per town

# Towns data
p = [0.5, 0.25, 0.35, 0.3, 0.5]  # Demand probabilities for towns 1 to 5
c = [60, 30, 50, 25, 100]  # Replenishment costs cj for towns 1 to 5
K = [200, 200, 200, 200, 200]  # Extra visit costs Kj for towns 1 to 5

horizon = 5  # Number of days

# Initialize transitions and rewards lists
transitions = np.zeros((num_states, num_actions, num_states))
rewards = np.zeros((num_states, num_actions))

for state in range(num_states):
    s = state % (M + 1)  # Spare parts in the repairman's inventory (0 to M)
    j = state // (M + 1)  # Current town (0 to N-1)
    j_next = (j+1) % N # Next town
    pj = p[j_next]
    demand_probs = [binom.pmf(k, 2, pj) for k in range(max_demand + 1)]  # Demands 0, 1, 2

    for a in range(num_actions):
        # Initialize expected cost and transition probabilities
        expected_cost = 0.0
        trans_probs = np.zeros(num_states)

        for d in range(max_demand + 1):  # Demands 0 to 2
            prob_d = demand_probs[d]
            spare_parts_used = min(s, d)
            extra_visit = 1 if d > s else 0

            # Spare parts after demand
            s_after_demand = s - spare_parts_used

            # Spare parts at the start of next day
            if a == 1:  # Replenish to full capacity
                s_next = M
            else:  # Do not replenish
                s_next = s_after_demand

            # Next state
            state_next = s_next + j_next * (M + 1)

            # Transition probability
            trans_probs[state_next] += prob_d

            # Immediate cost
            extra_visit_cost = K[j_next] if extra_visit else 0
            replenishment_cost = c[j] if a == 1 else 0

            expected_cost += prob_d * (extra_visit_cost + replenishment_cost)

        # Normalize transition probabilities (ensure they sum to 1)
        total_prob = np.sum(trans_probs)
        if total_prob > 0:
            trans_probs /= total_prob
        else:
            # This should not happen; if it does, assign equal probabilities
            trans_probs[:] = 1.0 / num_states

        # Assign the transition probabilities
        transitions[state, a, :] = trans_probs

        # Assign the expected reward
        rewards[state, a] = -expected_cost 


# Initialize and solve the MDP
# mdp = FiniteHorizonMDP(num_states, num_actions, transitions, rewards, horizon)
# V, policy, dV = mdp.solve()

# # Print the optimal policy
# print("Value:")
# print(V)
# print("\nPolicy:")
# print(policy)
# print("\nValue per step:")
# print(dV)

ihmdp = InfiniteHorizonMDP(num_states, num_actions, transitions, rewards, 0.9)
V_v, policy_v = ihmdp.value_iteration()
print("Value:")
print(V_v)
print("\nPolicy:")
print(policy_v)

V_p, policy_p = ihmdp.policy_iteration()
print("Value:")
print(V_p)
print("\nPolicy:")
print(policy_p)


Iteration 1: delta = 15000.000000000005
V = [ -87.5  -12.5    0.     0.     0.     0.  -115.5  -24.5    0.     0.
    0.     0.  -102.   -18.     0.     0.     0.     0.  -150.   -50.
    0.     0.     0.     0.  -150.   -50.     0.     0.     0.     0. ]
Iteration 2: delta = 9375.0
V = [-156.25      -71.63125   -14.765625   -1.378125    0.          0.
 -157.05      -56.95      -18.6165     -1.9845      0.          0.
 -162.2       -69.8       -31.05       -4.05        0.          0.
 -190.        -80.        -25.        -11.25        0.          0.
 -243.75     -116.875     -25.3125     -2.8125      0.          0.      ]
Iteration 3: delta = 393.1631766912465
V = [-1.63125000e+02 -7.96631250e+01 -3.89558531e+01 -1.06289719e+01
 -1.71694687e+00 -1.11628125e-01 -1.61205000e+02 -6.01950000e+01
 -3.18616500e+01 -2.21488875e+01 -5.08173750e+00 -4.46512500e-01
 -1.68220000e+02 -7.49800000e+01 -5.31050000e+01 -2.12962500e+01
 -6.27750000e+00 -9.11250000e-01 -1.94000000e+02 -8.30000000e+01
 -

### Exercise 4.4

In [6]:
import numpy as np
from MDP import *

# Define parameters.
num_states = 5 # States 1 to 4 and 0 is the obsorbing state
num_actions = 2  # Actions: 0 (do not exit), 1 (exit)

# Reward remaining in the system
r = [i for i in range(1, num_states+1)]

# Transition probabilities
P = [[0.3, 0.4, 0.2, 0.1],
     [0.2, 0.3, 0.5, 0.0],
     [0.1, 0.0, 0.8, 0.1],
     [0.4, 0.0, 0.0, 0.6]]

# Initialize transitions and rewards lists
transitions = np.zeros((num_states, num_actions, num_states))
rewards = np.zeros((num_states, num_actions))


def solve(R):
    for state in range(num_states):
        if state == 0:
            # From state 0, any action leads to state 0 with probability 1.0
            transitions[state, 0:2, 0] = 1.0
            rewards[state, 0:2] = 0
        else:
            # From states 1 to 4, action 0 leads to states 1 to 4 with probabilities defined by P
            transitions[state, 0, 1:5] = P[state-1]
            rewards[state, 0] = r[state-1]
            # From states 1 to 4, action 1 leads to state 0 with probability 1.0
            transitions[state, 1, 0] = 1.0
            rewards[state, 1] = R

    mdp = InfiniteHorizonMDP(num_states, num_actions, transitions, rewards, discount_factor=0.9)
    V, policy = mdp.value_iteration_discount(silence=True)
    return V, policy

# Find the minimum R that makes the agent exit the system at state 2
Rh = 1000
Rl = 20
iteration = 0
# Bisection method
while abs(Rh - Rl) > 1e-6:
    iteration += 1
    R = (Rh + Rl) / 2
    V, policy = solve(R)
    print(f"Iteration {iteration}: R = {R}, V = {V}")
    if policy[2] == 1:
        Rh = R
    else:
        Rl = R

print(f"R = {R}")
print(f"V = {V}")
print(f"policy = {policy}")

    

Iteration 1: R = 510.0, V = [  0. 510. 510. 510. 510.]
Iteration 2: R = 265.0, V = [  0. 265. 265. 265. 265.]
Iteration 3: R = 142.5, V = [  0.  142.5 142.5 142.5 142.5]
Iteration 4: R = 81.25, V = [ 0.   81.25 81.25 81.25 81.25]
Iteration 5: R = 50.625, V = [ 0.    50.625 50.625 50.625 50.625]
Iteration 6: R = 35.3125, V = [ 0.         35.3125     35.3125     35.3125     36.33142309]
Iteration 7: R = 27.65625, V = [ 0.         27.65625    27.65625    29.35561104 30.33967387]
Iteration 8: R = 23.828125, V = [ 0.         24.07661128 25.50777964 27.30439256 27.5380272 ]
Iteration 9: R = 25.7421875, V = [ 0.         25.7421875  26.50690099 28.25889714 28.84171193]
Iteration 10: R = 26.69921875, V = [ 0.         26.69921875 27.080939   28.80728512 29.59069291]
Iteration 11: R = 27.177734375, V = [ 0.         27.17773438 27.36789072 29.08141182 29.96518338]
Iteration 12: R = 27.4169921875, V = [ 0.         27.41699219 27.51140284 29.21851143 30.15242862]
Iteration 13: R = 27.53662109375, V 

### Exercise 5.1

In [7]:
import numpy as np
from MDP import *

# Define states and actions.
B = 5
num_states = B+1 # Queue length B
num_actions = 5 # Number of speeds K

# Data
speed = np.arange(1, num_actions+1)
lambd = 10
mu = 1
operation_cost = speed * 2
lost_cost = 100

# Initialize transitions and rewards lists
transitions = np.zeros((num_states, num_actions, num_states))
rewards = np.zeros((num_states, num_actions))
sojourntime = np.zeros((num_states, num_actions))

for s in range(num_states):
    for a in range(num_actions):
        # Compute the probability of transitioning to each next state.
        if 1 <= s < num_states - 1:
            transitions[s, a, s+1] = lambd/(lambd + mu * speed[a])
            transitions[s, a, s-1] = mu * speed[a]/(lambd + mu * speed[a])
            sojourntime[s, a] = 1/(lambd + mu * speed[a])
        elif s == 0:
            transitions[s, a, s+1] = 1
            sojourntime[s, a] = 1/lambd
        else:
            transitions[s, a, s] = lambd/(lambd + mu * speed[a])
            transitions[s, a, s-1] = mu * speed[a]/(lambd + mu * speed[a])
            sojourntime[s, a] = 1/(lambd + mu * speed[a])

        

        rewards[s, a] = -operation_cost[a] * sojourntime[s, a] - lost_cost * lambd * (s == B) * sojourntime[s, a]


# Initialize and solve the MDP.
tau = min(sojourntime.flatten()) * 0.9
smdp = SemiMDP(num_states, num_actions, transitions, sojourntime, rewards, tau=tau)
V, policy, avg_reward = smdp.value_iteration()
print("Value:")
print(V)
print("\nPolicy:")
print(policy)
print("\nTau:")
print(sojourntime)
print(f"\n Rewards: {rewards}")
print(f"\n Speed: {speed}")
print(f"\n Operation Cost: {operation_cost}")
for a in range(num_actions):
    print(f"\n Action: {transitions[:, a, :]}")  

print(f"\n Average Reward: {avg_reward}")

Iteration 1: delta = 497.5124378109453
V = [   -2.    -2.    -2.    -2.    -2. -1002.]
Iteration 2: delta = 352.2388059701493
V = [   -4.    -4.    -4.    -4.  -604. -1712.]
Iteration 3: delta = 336.1194029850744
V = [   -6.     -6.     -6.   -366.  -1098.8 -2389.6]
Iteration 4: delta = 308.8358208955223
V = [   -8.      -8.    -224.    -707.68 -1663.44 -3012.36]
Iteration 5: delta = 300.1611940298506
V = [  -10.     -139.6    -459.408 -1146.032 -2196.064 -3617.684]
Iteration 6: delta = 6.31508085746521
V = [  -89.76    -302.6048  -785.44   -1580.064  -2744.0264 -4201.198 ]
Iteration 7: delta = 3.4162218517744116
V = [ -219.46688  -538.45248 -1127.36384 -2050.05424 -3279.14064 -4774.04652]
Iteration 8: delta = 1.9034864904776265
V = [ -412.85824   -806.103616 -1514.304672 -2520.69896  -3817.358248
 -5335.574756]
Iteration 9: delta = 1.33044004527173
V = [ -650.8054656 -1123.0506368 -1915.680928  -3006.7762464 -4349.2903664
 -5890.1098036]
Iteration 10: delta = 0.9195743285124421
V = [ 

### Exercise 5.2

In [8]:
import numpy as np
from itertools import product
from MDP import *

# Warehouse capacity
K = 4

# Rates
mu = 1      # Manufacturing rate
lambd = 1   # Demand arrival rate

# Rewards and costs
r = 200        # Revenue per satisfied demand
h = 2        # Holding cost per item per time unit
c = 100        # Operating cost per time unit when machine is on
s_cost = 50   # Switching on cost

# States: (inventory_level, machine_status)
# inventory_level: 0 to K
# machine_status: 0 (Off), 1 (On)

inventory_levels = list(range(K + 1))
machine_statuses = [0, 1]
all_states = list(product(inventory_levels, machine_statuses))
num_states = len(all_states)

# Mapping from state tuple to index and vice versa
state_to_index = {state: idx for idx, state in enumerate(all_states)}
index_to_state = {idx: state for idx, state in enumerate(all_states)}

# Actions:
num_actions = 2

transitions = np.zeros((num_states, num_actions, num_states))
rewards = np.zeros((num_states, num_actions))
sojourntime = np.zeros((num_states, num_actions))

for s_idx, (inv, status) in enumerate(all_states):
    for a in range(num_actions):
        # Determine the action effect
        if a == 0:
            # Action 0: Keep current status
            new_status = status
            switching_cost = 0.0
        else:
            # Action 1: Toggle status
            new_status = 1 - status
            switching_cost = s_cost if new_status == 1 else 0.0

        # Define the event rates based on new_status
        if new_status == 1:
            rate_total = mu + lambd
        else:
            rate_total = lambd


        sojourn_time = 1 / rate_total

        sojourntime[s_idx, a] = sojourn_time

        holding_cost = h * inv * sojourn_time
        operating_cost = c * sojourn_time if new_status == 1 else 0.0

        # Initialize expected revenue
        expected_revenue = 0.0

        # Determine possible events and transitions
        if new_status == 1:
            # Machine is On: two possible events
            # 1. Manufacturing (rate mu)
            # 2. Demand arrival (rate lambd)
            # Probability of manufacturing: mu / (mu + lambd)
            # Probability of demand: lambd / (mu + lambd)

            # Manufacturing event
            prob_manu = mu / rate_total
            # If inventory < K, add one item; else, discard
            if inv < K:
                next_inv_manu = inv + 1
            else:
                next_inv_manu = inv  # Item discarded
            next_state_manu = (next_inv_manu, new_status)
            next_idx_manu = state_to_index[next_state_manu]
            transitions[s_idx, a, next_idx_manu] += prob_manu

            # Revenue is not affected by manufacturing
            reward_manu = - (holding_cost + operating_cost + switching_cost)

            # Demand arrival event
            prob_demand = lambd / rate_total
            if inv > 0:
                next_inv_demand = inv - 1
                expected_revenue += r  # Revenue from satisfied demand
            else:
                next_inv_demand = inv  # Demand lost
            next_state_demand = (next_inv_demand, new_status)
            next_idx_demand = state_to_index[next_state_demand]
            transitions[s_idx, a, next_idx_demand] += prob_demand

            # Reward for demand event
            reward_demand = - (holding_cost + operating_cost + switching_cost) + expected_revenue

            # Assign rewards
            rewards[s_idx, a] = prob_manu * reward_manu + prob_demand * reward_demand

        else:
            # Machine is Off: only demand arrival
            prob_demand = 1.0
            if inv > 0:
                next_inv_demand = inv - 1
                expected_revenue += r  # Revenue from satisfied demand
            else:
                next_inv_demand = inv  # Demand lost
            next_state_demand = (next_inv_demand, new_status)
            next_idx_demand = state_to_index[next_state_demand]
            transitions[s_idx, a, next_idx_demand] += prob_demand

            # Reward for demand event
            reward_demand = - (holding_cost + operating_cost + switching_cost) + expected_revenue

            # Assign rewards
            rewards[s_idx, a] = prob_demand * reward_demand


# Initialize and solve the MDP.
tau = min(sojourntime.flatten()) * 0.9
smdp = SemiMDP(num_states, num_actions, transitions, sojourntime, rewards, tau=tau)
V, policy, avg_reward = smdp.value_iteration()

print("\nStates:")
print(all_states)
print(f"\n Rewards: {rewards}")
for a in range(num_actions):
    print(f"\n Action: {transitions[:, a, :]}")  

print(f"\n Average Reward: {avg_reward}")
print("\nValue:")
print(V)
print("\nPolicy:")
print(policy)
print("\nTau:")
print(sojourntime)

# Print the optimal policy
for s_idx, (inv, status) in enumerate(all_states):
    print(f"Inventory: {inv}, Status: {status}")
    action = "Toggle" if policy[s_idx] == 1 else "Keep"
    print(f"Optimal Policy: {action}")


Iteration 1: delta = 19800.0
V = [  0.   0. 198. 198. 196. 196. 194. 194. 192. 192.]
Iteration 2: delta = 19690.000000000004
V = [  0.    0.  306.9 306.9 392.9 392.9 388.9 388.9 384.9 384.9]
Iteration 3: delta = 19580.000000000007
V = [  0.     38.105 366.795 366.795 550.2   550.2   584.7   584.7   578.7
 578.7  ]
Iteration 4: delta = 19470.000000000004
V = [  0.       86.0155  399.73725 399.73725 663.66775 663.66775 763.175
 763.175   773.4     773.4    ]
Iteration 5: delta = 9.337946532309047
V = [ 18.5887375 127.1902875 417.8554875 475.3311875 740.899025  740.899025
 912.3967375 912.3967375 960.79875   960.79875  ]
Iteration 6: delta = 8.281867090786033
V = [  72.9935375   183.8536925   436.18545     536.17330938  791.52943313
  794.56746875 1029.22276688 1029.22276688 1131.01784438 1131.01784438]
Iteration 7: delta = 3.0493387206090468
V = [ 131.31150459  242.39752009  481.90806756  591.9068535   827.62464072
  879.88498119 1116.26076669 1116.26076669 1277.2100595  1277.2100595 ]
I